In [ ]:
!pip install selenium
!apt install chromium-chromedriver
!pip install requests-html

In [ ]:
import re
import requests
import time
import pandas as pd
from requests_html import HTML
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [ ]:
australia = "Australia"
brazil = "Brazil (Brasil)"
canada = "Canada"
china = "China (中国大陆)"
france = "France"
germany = "Germany (Deutschland)"
india = "India"
italy = "Italy (Italia)"
japan = "Japan (日本)"
mexico = "Mexico (México)"
netherlands = "Netherlands (Nederland)"
saudi_arabia = "Saudi Arabia"
singapore = "Singapore"
spain = "Spain (España)"
sweden = "Sweden (Sverige)"
turkey = "Turkey (Türkiye)"
united_arab_emirates = "United Arab Emirates"
united_kingdom = "United Kingdom"
united_states = "United States"

amazon_country_int_codes = {1: australia,
                            2: brazil,
                            3: canada,
                            4: china,
                            5: france,
                            6: germany,
                            7: india,
                            8: italy,
                            9: japan,
                            10: mexico,
                            11: netherlands,
                            12: saudi_arabia,
                            13: singapore,
                            14: spain,
                            15: sweden,
                            16: turkey,
                            17: united_arab_emirates,
                            18: united_kingdom,
                            19: united_states}

amazon_country_links = {australia: "https://www.amazon.com.au",
                        brazil: "https://www.amazon.com.br",
                        canada: "https://www.amazon.ca",
                        china: "https://www.amazon.cn",
                        france: "https://www.amazon.fr",
                        germany: "https://www.amazon.de",
                        india: "https://www.amazon.in",
                        italy: "https://www.amazon.it",
                        japan: "https://www.amazon.co.jp",
                        mexico: "https://www.amazon.com.mx",
                        netherlands: "https://www.amazon.nl",
                        saudi_arabia: "https://www.amazon.sa",
                        singapore: "https://www.amazon.sg",
                        spain: "https://www.amazon.es",
                        sweden: "https://www.amazon.se",
                        turkey: "https://www.amazon.com.tr",
                        united_arab_emirates: "https://www.amazon.ae",
                        united_kingdom: "https://www.amazon.co.uk",
                        united_states: "https://www.amazon.com"}

In [ ]:
import json
### print json  (global helper)
def print_json(js):
    print(json.dumps(js, indent=2))
for entry in amazon_country_links:
    print(str(entry)+": "+str(amazon_country_links[entry]))
print()
for entry in amazon_country_int_codes:
    print(str(entry)+": "+str(amazon_country_int_codes[entry]))
print()
print_json(amazon_country_links)

Australia: https://www.amazon.com.au
Brazil (Brasil): https://www.amazon.com.br
Canada: https://www.amazon.ca
China (中国大陆): https://www.amazon.cn
France: https://www.amazon.fr
Germany (Deutschland): https://www.amazon.de
India: https://www.amazon.in
Italy (Italia): https://www.amazon.it
Japan (日本): https://www.amazon.co.jp
Mexico (México): https://www.amazon.com.mx
Netherlands (Nederland): https://www.amazon.nl
Saudi Arabia: https://www.amazon.sa
Singapore: https://www.amazon.sg
Spain (España): https://www.amazon.es
Sweden (Sverige): https://www.amazon.se
Turkey (Türkiye): https://www.amazon.com.tr
United Arab Emirates: https://www.amazon.ae
United Kingdom: https://www.amazon.co.uk
United States: https://www.amazon.com

1: Australia
2: Brazil (Brasil)
3: Canada
4: China (中国大陆)
5: France
6: Germany (Deutschland)
7: India
8: Italy (Italia)
9: Japan (日本)
10: Mexico (México)
11: Netherlands (Nederland)
12: Saudi Arabia
13: Singapore
14: Spain (España)
15: Sweden (Sverige)
16: Turkey (Türki

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver', options=options)

In [ ]:
def generate_links_single_country(amazon_country_int_code):
    if amazon_country_int_code not in [4, 11, 15]:
        search_country = amazon_country_int_codes[amazon_country_int_code]
        country_link = amazon_country_links[search_country]
        print(f"AMAZON @ {search_country}:", country_link)
        print()

        todays_deals_url = f"{country_link}/gp/goldbox?ref_=nav_cs_gb"
        print("  Today's Deals:", todays_deals_url)
        print()

        best_sellers_multi_categories_url = f"{country_link}/gp/bestsellers/?ref_=nav_cs_bestsellers"
        print("  Best Sellers:", best_sellers_multi_categories_url)
        print()

        result_bestseller_category_links_dict = {}
        time.sleep(1)
        final_page_links = {}
        found_page_links = False
        while (not found_page_links):
            time.sleep(1.5)
            driver.get(best_sellers_multi_categories_url)
            time.sleep(1.5)
            body_el = driver.find_element_by_css_selector("body")
            time.sleep(1.5)
            html_str = body_el.get_attribute("innerHTML")
            time.sleep(1.5)
            html_obj = HTML(html=html_str)
            time.sleep(1.5)
            page_links = [f"{country_link}{x}" for x in html_obj.links if x.startswith("/")]
            page_links = [x for x in page_links if "&ref_=nav_cs_" in x]
            if page_links != []:
                found_page_links = True
        for url in page_links:
          temp = url.split("&re", 1)[1]
          bestseller_category_temp = temp.split("_", 3)[3]
          bestseller_category = bestseller_category_temp.split("_", bestseller_category_temp.count("_"))[0]
          url = url.split("_", url.count("_"))[0]
          if bestseller_category not in ["help", "sell", "freeshipping", "customerservice", "coupons", "shoppertoolkit", "registry", "bestsellers"]:
              final_page_links[bestseller_category] = url
        if len(final_page_links) != 1:
            result_bestseller_category_links_dict[search_country] = final_page_links
        for key in sorted(result_bestseller_category_links_dict):
            print(f"  Best Sellers & Links: ")
            for value in sorted(result_bestseller_category_links_dict[key]):
                print("    "+str(value)+": "+str(result_bestseller_category_links_dict[key][value]))
            print()

    else:
        result_bestseller_category_links_dict = {}
        search_country = amazon_country_int_codes[amazon_country_int_code]
        country_link = amazon_country_links[search_country]
        print(f"Amazon @ {search_country}:", country_link)
        print()

        todays_deals_url = f"{country_link}/gp/goldbox?ref_=nav_cs_gb"
        print("  Today's Deals:", todays_deals_url)
        print()

        best_sellers_multi_categories_url = f"{country_link}/gp/bestsellers/?ref_=nav_cs_bestsellers"
        print("  Best Sellers:", best_sellers_multi_categories_url)
        print()

    return result_bestseller_category_links_dict


In [ ]:
links =[]
for i in range(1, 20):
    result = generate_links_single_country(i)
    if result != {}:
        links.append(result)

AMAZON @ Australia: https://www.amazon.com.au

  Today's Deals: https://www.amazon.com.au/gp/goldbox?ref_=nav_cs_gb

  Best Sellers: https://www.amazon.com.au/gp/bestsellers/?ref_=nav_cs_bestsellers

  Best Sellers & Links: 
    automotive: https://www.amazon.com.au/Automotive/b/?ie=UTF8&node=4851453051&ref
    beauty: https://www.amazon.com.au/gp/browse.html?node=4851567051&ref
    books: https://www.amazon.com.au/books-store/b/?ie=UTF8&node=4851626051&ref
    electronics: https://www.amazon.com.au/electronics-store/b/?ie=UTF8&node=4851799051&ref
    fashion: https://www.amazon.com.au/Amazon-Fashion/b/?ie=UTF8&node=4851856051&ref
    gc: https://www.amazon.com.au/gift-cards/b/?ie=UTF8&node=5264602051&ref
    hi: https://www.amazon.com.au/tools-home-improvement-DIY/b/?ie=UTF8&node=4852033051&ref
    home: https://www.amazon.com.au/home-decor-furniture-bedding/b/?ie=UTF8&node=4851975051&ref
    hpc: https://www.amazon.com.au/gp/browse.html?node=4851917051&ref
    kindle: https://www.ama

In [ ]:
 import csv
 with open("international_amazon_bestseller_categories.csv", 'w', encoding='utf-8') as outputfile:  
    writer = csv.writer(outputfile)
    writer.writerow(["Country", "Bestseller Category", "URL"])
    for country in links:
      for key in sorted(country):
          categories_dict = country[key]
          for category in sorted(categories_dict):
              writer.writerow([key, category, categories_dict[category]])